# Open Weather Map
Exploration of openweathermap.org API.
## Prerequisits

In [1]:
import toml
import requests
import pandas as pd

def del_lst(x):
    return x[0]

conf = toml.load('../../conf.toml')
owm = conf['owm']

url = 'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lng}' \
      '&units=metric&APPID={key}'
url = url.format(lat=owm['lat'], lng=owm['lng'], key=owm['key'])

## API
Get data from API and safe json file.

In [2]:
response = requests.get(url)
data = response.json()
data.keys()

dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current', 'minutely', 'hourly', 'daily'])

## Current Weather
- Timezone is UTC
- temp, feels_like, dew_point: °C
- pressure: hPa
- humidity, clouds: %
- uvi: UV Index
- visibility: m
- wind_speed: m / s
- wind_deg: °

In [3]:
keys = ['timezone', 'timezone_offset', 'current']
time = ['dt', 'sunrise', 'sunset']

cur = pd.DataFrame(list(map(data.get, keys)), keys).T
cur = cur.join(pd.DataFrame(cur.current[0], [0]))
cur = cur.join(pd.DataFrame(cur.weather[0], [0]))
cur = cur.drop(columns=['current', 'weather'])

cur['timezone_offset'] = pd.to_timedelta(cur.timezone_offset, 's')
cur[time] = cur[time].apply(pd.to_datetime, unit='s')
cur = cur.set_index('dt')
cur

,timezone,timezone_offset,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,
2021-05-24 20:38:26,Europe/Berlin,0 days 02:00:00,2021-05-24 03:15:28,2021-05-24 19:37:04,10.33,9.85,1006,93,9.25,0,75,10000,1.54,190,803,Clouds,broken clouds,04n


### Alerts

In [4]:
# ale = pd.DataFrame(data['alerts'], ['german', 'english'])
#
# time = ['start', 'end']
#
# ale[time] = ale[time].apply(pd.to_datetime, unit='s')
# ale

## Minutely Data
- 1 hour
- 60 observations
- precipitation: mm

In [5]:
mnt = pd.DataFrame(data['minutely'])
mnt = mnt.set_index('dt')
mnt.index = pd.to_datetime(mnt.index, unit='s')
mnt

,precipitation
dt,
2021-05-24 20:39:00,0
2021-05-24 20:40:00,0
2021-05-24 20:41:00,0
2021-05-24 20:42:00,0
2021-05-24 20:43:00,0
...,...
2021-05-24 21:35:00,0
2021-05-24 21:36:00,0
2021-05-24 21:37:00,0


## Hourly Data
- 2 days
- 48 observations
- wind_gust: metre/sec
- pop: %
- rain: mm

In [6]:
hr = pd.DataFrame(data['hourly'])

hr = hr.set_index('dt')
hr.index = pd.to_datetime(hr.index, unit='s')

s = hr.rain.notna()
hr['precipitation'] = 0
hr.loc[s, 'precipitation'] = hr.loc[s, 'rain'].apply(lambda x: x.get('1h'))
hr = hr.drop(columns='rain')

hr = hr.join(pd.DataFrame(hr.weather.apply(del_lst).tolist(), hr.index))
hr = hr.drop(columns='weather')
hr

,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,precipitation,id,main,description,icon
dt,,,,,,,,,,,,,,,,,
2021-05-24 20:00:00,10.22,9.75,1006,94,9.30,0.00,80,10000,2.96,153,4.99,0.47,0.00,803,Clouds,broken clouds,04n
2021-05-24 21:00:00,10.33,9.85,1006,93,9.25,0.00,75,10000,2.92,146,4.19,0.26,0.00,803,Clouds,broken clouds,04n
2021-05-24 22:00:00,10.05,9.57,1006,94,9.13,0.00,80,10000,3.08,155,5.37,0.17,0.00,803,Clouds,broken clouds,04n
2021-05-24 23:00:00,9.59,7.87,1006,95,8.83,0.00,85,10000,3.27,155,6.04,0.19,0.00,804,Clouds,overcast clouds,04n
2021-05-25 00:00:00,8.88,6.78,1007,96,8.28,0.00,88,10000,3.69,155,7.96,0.26,0.13,500,Rain,light rain,10n
2021-05-25 01:00:00,8.26,5.84,1006,97,7.81,0.00,95,10000,4.05,176,9.01,0.36,0.00,804,Clouds,overcast clouds,04n
2021-05-25 02:00:00,7.80,5.08,1006,98,7.41,0.00,92,10000,4.45,193,8.94,0.30,0.00,804,Clouds,overcast clouds,04n
2021-05-25 03:00:00,7.94,5.02,1006,95,7.19,0.00,95,10000,4.98,207,9.70,0.36,0.22,500,Rain,light rain,10n
2021-05-25 04:00:00,7.73,4.66,1006,95,7.02,0.00,96,10000,5.22,200,10.38,0.43,0.00,804,Clouds,overcast clouds,04d


## Daily Data
- 1 week
- 8 observations

In [7]:
time = ['dt', 'sunrise', 'sunset']
dly = pd.DataFrame(data['daily'])
dly[time] = dly[time].apply(pd.to_datetime, unit='s')

dly = dly.join(pd.DataFrame(dly.temp.to_list()))
dly = dly.join(pd.DataFrame(dly.feels_like.to_list()).add_suffix('_feels_like'))
dly = dly.join(pd.DataFrame(dly.weather.apply(del_lst).to_list()))

dly = dly.drop(columns=['temp', 'feels_like', 'weather'])
dly = dly.set_index('dt')
dly

,sunrise,sunset,moonrise,moonset,moon_phase,pressure,humidity,dew_point,wind_speed,wind_deg,...,eve,morn,day_feels_like,night_feels_like,eve_feels_like,morn_feels_like,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,,,,
2021-05-24 11:00:00,2021-05-24 03:15:28,2021-05-24 19:37:04,1621875240,1621823460,0.42,1008,82,9.94,7.03,187,...,10.81,9.53,12.51,9.85,10.40,6.81,500,Rain,light rain,10d
2021-05-25 11:00:00,2021-05-25 03:14:14,2021-05-25 19:38:31,1621967340,1621911000,0.46,1008,91,7.57,7.21,207,...,8.01,7.80,5.57,4.26,4.52,4.42,501,Rain,moderate rain,10d
2021-05-26 11:00:00,2021-05-26 03:13:02,2021-05-26 19:39:57,1622059380,1621998780,0.50,1006,77,7.60,9.99,236,...,9.57,7.98,10.82,5.56,6.27,4.46,501,Rain,moderate rain,10d
2021-05-27 11:00:00,2021-05-27 03:11:52,2021-05-27 19:41:20,1622151000,1622087100,0.54,1016,79,8.90,9.12,283,...,10.77,9.21,11.79,5.16,10.20,5.74,500,Rain,light rain,10d
2021-05-28 11:00:00,2021-05-28 03:10:46,2021-05-28 19:42:41,0,1622176260,0.58,1024,67,7.63,5.70,4,...,11.48,10.67,12.78,6.24,10.90,10.09,500,Rain,light rain,10d
2021-05-29 11:00:00,2021-05-29 03:09:42,2021-05-29 19:44:01,1622241660,1622266380,0.61,1027,72,9.09,5.27,357,...,11.37,11.58,13.51,6.91,10.76,10.99,500,Rain,light rain,10d
2021-05-30 11:00:00,2021-05-30 03:08:41,2021-05-30 19:45:18,1622331180,1622357280,0.65,1024,72,9.82,5.67,29,...,14.06,10.96,14.16,9.39,13.66,10.25,500,Rain,light rain,10d
2021-05-31 11:00:00,2021-05-31 03:07:43,2021-05-31 19:46:33,1622419680,1622448600,0.69,1020,79,10.03,4.08,60,...,14.07,11.14,12.90,11.15,13.78,10.42,500,Rain,light rain,10d
